In [3]:
import cv2 as cv
import numpy as np

#constants
lookUpTable = np.empty((1,256), np.uint8)
for i in range(256):
    lookUpTable[0,i] = np.clip(pow(i / 255.0, 23/100) * 255.0, 0, 255)
threshold1 = 50
threshold2 = 225
gamma = 23
kernel1 = 4
kernel2 = 3
arcRatio = 0.08
def empty(a):
    pass
# 0: top left, 1: top right, 2: bottom left, 3: bottom right
def reorder (a):
    b = np.zeros_like(a)
    a = a.reshape((4,2))
    add = a.sum(1)
    b[0] = a[np.argmin(add)]
    b[3] = a[np.argmax(add)]
    diff = np.diff(a, axis = 1)
    b[1] = a[np.argmin(diff)]
    b[2] = a[np.argmax(diff)]
    return b

# tries to set the output image size with max_x - min_x, and max_y - min_y
def size(a):
    a = a.reshape((4,2))
    max = np.argmax(a, axis=0) 
    min = np.argmin(a, axis=0)
    return int(a[max[0]][0] - a[min[0]][0]), int(a[max[1]][1] - a[min[1]][1])

# warps the image from given points
def warpImage(img, pts1):
    width, height = size(pts1)
    pts2 = np.float32([[0,0], [width,0], [0,height],[width,height]])
    M = cv.getPerspectiveTransform(pts1, pts2)
    dst = cv.warpPerspective(img, M, (width,height))
    return dst

# gets the required points to be warped
def getPoints(img):
    frameHeight, frameWidth, channels = img.shape
    calculatedArea = frameWidth*frameHeight
    img = cv.GaussianBlur(img, (1,1),1)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.LUT(img, lookUpTable)
    img = cv.Canny(img, threshold1,threshold2)
    img = cv.dilate(img, np.ones((kernel1,kernel1)), iterations = 2)
    img = cv.erode(img, np.ones((kernel2,kernel2)), iterations = 2)
    contours, hierarchy = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    count = 0
    for cnt in contours:
        area = cv.contourArea(cnt)
        peri = cv.arcLength(cnt, True)
        approx = cv.approxPolyDP(cnt, arcRatio *peri, True)
        if(area > calculatedArea*.25 and len(approx) == 4):
            count = count +1
            if (count == 1):
                outCon = cnt
            elif (area < cv.contourArea(outCon)):
                outCon = cnt
    if count > 0:
        area = cv.contourArea(outCon)
        peri = cv.arcLength(outCon, True)
        approx = cv.approxPolyDP(outCon, arcRatio * peri, True)
        approxReorder = reorder(approx)
        pts1 = np.float32(approxReorder)
        return pts1
    return np.float32([[0,0], [0,0], [0,0],[0,0]])
        
while True:
    # success, img = cap.read()
    img = cv.imread('test3'+ '.jpg')
    points = getPoints(img)
    output = warpImage(img, points)
    cv.imshow ("original", cv.resize(img, (0,0), None, .25, .25))
    cv.imshow("output", cv.resize(output, (0,0), None, .25, .25)) 
    if(cv.waitKey(1) & 0xFF == ord('q')):
        cv.destroyAllWindows()
        break



KeyboardInterrupt: 